In [3]:
import cv2
import face_recognition
import numpy as np

def analyzeFaces():
    # Initialize the camera
    cap = cv2.VideoCapture(0)  # 0 is typically the default camera

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Camera could not be accessed.")
        return {}

    # Capture a single frame
    ret, frame = cap.read()

    # Release the camera
    cap.release()

    # Check if the frame was captured successfully
    if not ret:
        print("Error: No frame captured.")
        return {}

    face_encodings = face_recognition.face_encodings(frame)

    # Dictionary to store face data
    face_data = {}

    # Generate face data
    for i, face_encoding in enumerate(face_encodings):
        # Assume each face ID is unique based on its encoding
        face_id = f"face_{i+1}"

        # Analyze the face to determine if it's looking at the camera (simple heuristic: if the face is in the central part of the frame)
        top, right, bottom, left = face_locations[i]
        face_center_x = (left + right) // 2
        face_center_y = (top + bottom) // 2
        frame_center_x, frame_center_y = frame.shape[1] // 2, frame.shape[0] // 2

        # Check if the center of the face is within 20% of the center of the frame to consider it looking at/near the camera
        if abs(face_center_x - frame_center_x) < frame.shape[1] * 0.2 and abs(face_center_y - frame_center_y) < frame.shape[0] * 0.2:
            face_data[face_id] = {
                "face_location": (top, right, bottom, left),
                "face_encoding": face_encoding.tolist(),  # Convert numpy array to list for serialization
                "looking_at_camera": True
            }
        else:
            face_data[face_id] = {
                "face_location": (top, right, bottom, left),
                "face_encoding": face_encoding.tolist(),
                "looking_at_camera": False
            }

    return face_data

In [4]:
# Example usage
faces = analyzeFaces()

In [ ]:
import cv2
import time
import logging
import datetime

# Placeholder function for face detection and identification
def identify_faces(frame):
    # Your face detection and identification logic here
    # Return a dictionary of detected face IDs with their positions
    # return {"face_id1": (x, y, w, h)}
    return

# Initialize system info retrieval and logging setup here (from previous steps)

faces_last_seen = {}  # Track when each face was last seen
face_absence_start = {}  # Track when a face is no longer seen

camera = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = camera.read()
        if not ret:
            break

        identified_faces = identify_faces(frame)
        current_time = time.time()
        
        # Update or add detected faces with the current timestamp
        for face_id in identified_faces:
            if face_id not in faces_last_seen:
                logging.info(f"Face {face_id} detected at {datetime.datetime.now()}.")
            faces_last_seen[face_id] = current_time
            if face_id in face_absence_start:
                del face_absence_start[face_id]  # Face reappeared; remove from absence tracking

        # Check for faces that are no longer visible
        for face_id in list(faces_last_seen):
            if face_id not in identified_faces:
                if face_id not in face_absence_start:
                    face_absence_start[face_id] = current_time  # Start tracking absence
                elif current_time - face_absence_start[face_id] > 900:  # 15 minutes in seconds
                    logging.info(f"Face {face_id} has been out of view for more than 15 minutes.")
                    del faces_last_seen[face_id]  # Remove from tracking
                    del face_absence_start[face_id]  # Reset absence tracking
        
        # Display the frame (optional)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    camera.release()
    cv2.destroyAllWindows()


In [6]:
def takePic():
    cap = cv2.VideoCapture(0)  # 0 is typically the default camera

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Camera could not be accessed.")
        return {}

    # Capture a single frame
    ret, frame = cap.read()

    # Release the camera
    cap.release()

    # Check if the frame was captured successfully
    if not ret:
        print("Error: No frame captured.")
        return {}
    return frame

In [10]:
import cv2
import face_recognition
frame4 = takePic()

In [11]:
cv2.imshow('frame', frame4)
cv2.waitKey(5000) # 5 seconds
cv2.destroyAllWindows() # close when done

In [13]:
def calculate_distance(image_height, face_size, avg_size):
    size_ratio = image_height/face_size
    plane_height = avg_size*size_ratio #total height of plane perpendicular to camera anngle where face is positioned

    z_distance = (plane_height/2)*1.327044822  # found using trigonometry
    
    return z_distance, plane_height

In [14]:
def determine_face_position(image_width, image_height, face_size_x, face_size_y, face_center_x, face_center_y, image_center_x, image_center_y):
    z_distance1, plane_height = calculate_distance(image_height, face_size_y, 0.12)
    z_distance2, plane_width = calculate_distance(image_width, face_size_x, 0.125)
    z_distance = (z_distance1 + z_distance2)/2

    # Change x and y to interpretable lengths(meters)
    world_y = (( face_center_y - image_center_y)/image_height) * plane_height
    world_x = (( face_center_x - image_center_x)/image_width) * plane_width
    face_position = [world_x, world_y, z_distance]  # Position is meters away from camera in each dimension

    return face_position

In [21]:
def isGazeClose(face_pos, vector, threshold):
    p = np.array(face_pos)  # Face position
    v = np.array(vector)  # Normalized direction vector

    # Calculate t for the closest point
    t = -np.dot(p, v)

    # Calculate the closest point
    closest_point = p + t * v
    print("Closest Point is:")
    print(closest_point)

    # Calculate the distance from the camera to this closest point
    distance = (closest_point[0]**2+closest_point[1]**2+closest_point[2]**2)**0.5

    print("Distance is:")
    print(distance)
    print("Threshold is:")
    print(threshold)

    # Determine if this distance is within the acceptable threshold
    if distance is not None and threshold is not None:
        return distance < threshold
    return None

In [15]:
def determine_threshold(confidence, z_distance):
    threshold = (1-confidence)*z_distance
    return threshold

In [16]:
import cv2
import numpy as np

def cameraSpying(image):
    if image is None:
        return "Input in CameraSpying is Null"

    # Detect face landmarks
    face_landmarks_list = face_recognition.face_landmarks(image)
    face_locations = face_recognition.face_locations(image)
    
    if not face_landmarks_list:
        return "No faces detected"

    # Get image dimensions and center
    img_height, img_width = image.shape[:2]
    img_center_x, img_center_y = img_width // 2, img_height // 2

    faces_dict = {}

    # Process each face found
    for face_landmarks, (top, right, bottom, left) in zip(face_landmarks_list, face_locations):
        
        face_center_x = (left + right) // 2
        face_center_y = (top + bottom) // 2
        face_size_y = bottom - top
        face_size_x = right - left
        
        isLooking = False
        
        # Extract eye regions and calculate pupil centers
        for eye_key in ['left_eye', 'right_eye']:
            eye_points = face_landmarks[eye_key]
            eye_array = np.array(eye_points, dtype=np.int32)

            # Calculate the eyeball center by finding the average of the eye corner landmarks
            leftmost = min(eye_points, key=lambda x: x[0])
            rightmost = max(eye_points, key=lambda x: x[0])
            topmost = min(eye_points, key=lambda x: x[1])
            bottommost = max(eye_points, key=lambda x: x[1])

            eye_center_x = (leftmost[0] + rightmost[0]) // 2
            eye_center_y = (topmost[1] + bottommost[1]) // 2
            
            # Create a mask to isolate the eye area
            mask = np.zeros((img_height, img_width), dtype=np.uint8)
            cv2.fillConvexPoly(mask, eye_array, 255)
            eye_region = cv2.bitwise_and(image, image, mask=mask)

            # Convert eye region to grayscale and apply GaussianBlur
            gray_eye = cv2.cvtColor(eye_region, cv2.COLOR_BGR2GRAY)
            blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)

            # Find the darkest point in the eye region, which is the pupil
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(blurred_eye, mask=mask)
            pupil_x, pupil_y = min_loc

            eyeFaceRatio = (1.15/12) # approximate ratio of eyeball radius to vertical face length
            eyeRadius = eyeFaceRatio * face_size_y
            
            change_in_z = (eyeRadius**2 - (pupil_y - eye_center_y)**2 - (pupil_x - eye_center_x)**2)**0.5

            slope_vector = [(pupil_x - eye_center_x)/eyeRadius, (pupil_y - eye_center_y)/eyeRadius, change_in_z/eyeRadius] #normalized to have length 1(pixel)
            print("Slope Vector:")
            print(slope_vector)
                  
            
            face_pos = determine_face_position(img_width, img_height, face_size_x, face_size_y, face_center_x, face_center_y, img_center_x, img_center_y)
            print("Face position is:")
            print(face_pos)
            
            confidence = 0.3 #adjust depending on what confidence you want for gaze
            threshold = determine_threshold(confidence, face_pos[2])
            
            looking = isGazeClose(face_pos, slope_vector, threshold) #boolean

            if looking is None:
                print("Gaze not found")
            
            if looking:
                isLooking = True

        faces_dict[1] = {"face_location" : (face_center_x, face_center_y, img_height, img_width), "Looking": isLooking}
    return faces_dict

In [22]:
cameraSpying(takePic())

Slope Vector:
[-1.0077204388459975, -0.03250711093051605, (7.879056949944455e-18+0.12867476492700036j)]
Face position is:
[0.017523364485981307, 0.020186915887850466, 0.14221290304299064]
Closest Point is:
[-0.00093291+0.01844049j  0.01959155+0.00059485j  0.14456755+0.00235666j]
Distance is:
(0.14472000352949335+0.0023158353656242037j)
Threshold is:
0.09954903213009345
Slope Vector:
[1.0727346607070296, 0.0650142218610321, (2.4106146599199328e-17+0.3936832499947408j)]
Face position is:
[0.017523364485981307, 0.020186915887850466, 0.14221290304299064]
Closest Point is:
[-0.00404971-0.06005902j  0.01887946-0.00363994j  0.16425398-0.00791711j]
Distance is:
(0.15402151483080156-0.007310117048639081j)
Threshold is:
0.09954903213009345


{1: {'face_location': (365, 294, 480, 640), 'Looking': False}}

In [26]:
cv2.imshow('frame', frame4)
cv2.waitKey(0) # until a key is pressed
cv2.destroyAllWindows() # close when done